<a href="https://colab.research.google.com/github/LuchoValr/Machine_Learning/blob/main/Proyecto3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importacion de librerias

In [375]:
import pandas as pd
from sklearn.impute import MissingIndicator
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

##Carga de datos

In [395]:
data=pd.read_csv('https://raw.githubusercontent.com/labeconometria/MLxE/main/proyectos1er/dataset_1.csv')
data.head()

,user_id,gender,wage,purchased,birth_day
0,217,Female,NaN,0,1983-08-06
1,226,Female,146500.0,1,1972-01-28
2,697,Male,NaN,1,1981-12-12
3,743,Female,NaN,0,1983-05-26
4,63,Female,140500.0,1,1967-11-02


##Exploración inicial

In [396]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1174 entries, 0 to 1173
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   user_id    1174 non-null   int64  
 1   gender     1174 non-null   object 
 2   wage       930 non-null    float64
 3   purchased  1174 non-null   int64  
 4   birth_day  1174 non-null   object 
dtypes: float64(1), int64(2), object(2)
memory usage: 46.0+ KB


In [397]:
data.describe()

,user_id,wage,purchased
count,1174.000000,930.000000,1174.000000
mean,499.988927,71948.387097,0.404600
std,287.648865,34112.616989,0.491024
min,1.000000,15000.000000,0.000000
25%,252.250000,45500.000000,0.000000
50%,499.500000,71250.000000,0.000000
75%,749.750000,89500.000000,1.000000
max,1000.000000,152500.000000,1.000000


##Revisión de duplicados

In [398]:
data['user_id'].duplicated().sum()

174

In [399]:
data = data.drop_duplicates().reset_index(drop=True)
data

,user_id,gender,wage,purchased,birth_day
0,217,Female,NaN,0,1983-08-06
1,226,Female,146500.0,1,1972-01-28
2,697,Male,NaN,1,1981-12-12
3,743,Female,NaN,0,1983-05-26
4,63,Female,140500.0,1,1967-11-02
...,...,...,...,...,...
995,305,Female,22500.0,1,1969-03-09
996,636,Male,76500.0,0,1970-08-26
997,320,Female,143500.0,1,1970-04-07
998,721,Male,148000.0,1,1993-11-23


In [400]:
data['gender'].value_counts()

Female    446
Male      423
woman      76
man        55
Name: gender, dtype: int64

In [401]:
data['gender'] = data['gender'].replace({'woman':'Female', 'man':'Male'})
data.head()

,user_id,gender,wage,purchased,birth_day
0,217,Female,NaN,0,1983-08-06
1,226,Female,146500.0,1,1972-01-28
2,697,Male,NaN,1,1981-12-12
3,743,Female,NaN,0,1983-05-26
4,63,Female,140500.0,1,1967-11-02


In [402]:
data['gender'].value_counts()

Female    522
Male      478
Name: gender, dtype: int64

##Transformación de tipos de datos

In [403]:
data = data.astype({'user_id':'int16','purchased':'int16','wage':'float32'})
data.head()

,user_id,gender,wage,purchased,birth_day
0,217,Female,NaN,0,1983-08-06
1,226,Female,146500.0,1,1972-01-28
2,697,Male,NaN,1,1981-12-12
3,743,Female,NaN,0,1983-05-26
4,63,Female,140500.0,1,1967-11-02


##Enriquecimiento de datos

In [404]:
age = data['birth_day'].str.split('-', expand=True)
age = age[0].astype('int16')
age

0      1983
1      1972
2      1981
3      1983
4      1967
       ... 
995    1969
996    1970
997    1970
998    1993
999    1984
Name: 0, Length: 1000, dtype: int16

In [405]:
data['age'] = 2023 - age
data = data.drop(['birth_day'], axis = 1)
data.head()

,user_id,gender,wage,purchased,age
0,217,Female,NaN,0,40
1,226,Female,146500.0,1,51
2,697,Male,NaN,1,42
3,743,Female,NaN,0,40
4,63,Female,140500.0,1,56


In [406]:
data['cat_age'] = pd.cut(data.age, bins=[0, 25, 30, 60, float('Inf')], labels=['teen', 'young','adult', 'old'])
data['cat_age'].value_counts()

adult    765
young    113
teen      77
old       45
Name: cat_age, dtype: int64

##Missing values

In [407]:
data.isna().sum()

user_id        0
gender         0
wage         214
purchased      0
age            0
cat_age        0
dtype: int64

In [408]:
indic = MissingIndicator()
data['indicator_wage'] = indic.fit_transform(data).astype('int16')
data.head()

,user_id,gender,wage,purchased,age,cat_age,indicator_wage
0,217,Female,NaN,0,40,adult,1
1,226,Female,146500.0,1,51,adult,0
2,697,Male,NaN,1,42,adult,1
3,743,Female,NaN,0,40,adult,1
4,63,Female,140500.0,1,56,adult,0


In [409]:
data = pd.get_dummies(data, drop_first=True)
age = pd.DataFrame(data['age'])
data = data.drop(['age'], axis=1)

#Estandarización
scaler = StandardScaler()
df= scaler.fit(data)
df_scaler = df.transform(data)

In [410]:
knn_imputer = KNNImputer(n_neighbors = 4)
data_scaled = pd.DataFrame(knn_imputer.fit_transform(df_scaler))
data_scaled.head()

,0,1,2,3,4,5,6,7
0,-0.982073,0.534210,-0.819903,1.91648,-0.956927,-0.356925,0.554247,-0.217072
1,-0.950896,2.158463,1.219657,-0.52179,-0.956927,-0.356925,0.554247,-0.217072
2,0.680696,-0.181191,1.219657,1.91648,1.045012,-0.356925,0.554247,-0.217072
3,0.840045,-0.469541,-0.819903,1.91648,-0.956927,-0.356925,0.554247,-0.217072
4,-1.515545,1.983262,1.219657,-0.52179,-0.956927,-0.356925,0.554247,-0.217072


In [411]:
data = pd.DataFrame(knn_imputer.fit_transform(data), columns=data.columns)
data.head()

,user_id,wage,purchased,indicator_wage,gender_Male,cat_age_young,cat_age_adult,cat_age_old
0,217.0,88125.0,0.0,1.0,0.0,0.0,1.0,0.0
1,226.0,146500.0,1.0,0.0,0.0,0.0,1.0,0.0
2,697.0,71000.0,1.0,1.0,1.0,0.0,1.0,0.0
3,743.0,79250.0,0.0,1.0,0.0,0.0,1.0,0.0
4,63.0,140500.0,1.0,0.0,0.0,0.0,1.0,0.0


##Outliers

In [412]:
isol = IsolationForest(random_state=0).fit(data_scaled)
data_scaled['isol_forest'] = isol.predict(data_scaled)
data_scaled['isol_forest'].value_counts()

-1    514
 1    486
Name: isol_forest, dtype: int64

In [413]:
isol = IsolationForest(random_state=0).fit(data)
data['isol_forest'] = isol.predict(data)
data['isol_forest'].value_counts()

-1    520
 1    480
Name: isol_forest, dtype: int64

##Clasificación de datos

In [414]:
data['cat_wage'] = pd.cut(data.wage, bins=[0, 5000, 10000, 50000, 100000, float('Inf')], labels=['low', 'low-medium', 'medium', 'medium-high', 'high'])
data.head()

,user_id,wage,purchased,indicator_wage,gender_Male,cat_age_young,cat_age_adult,cat_age_old,isol_forest,cat_wage
0,217.0,88125.0,0.0,1.0,0.0,0.0,1.0,0.0,-1,medium-high
1,226.0,146500.0,1.0,0.0,0.0,0.0,1.0,0.0,-1,high
2,697.0,71000.0,1.0,1.0,1.0,0.0,1.0,0.0,-1,medium-high
3,743.0,79250.0,0.0,1.0,0.0,0.0,1.0,0.0,-1,medium-high
4,63.0,140500.0,1.0,0.0,0.0,0.0,1.0,0.0,-1,high


##Porcentajes

In [415]:
data_table = data.copy()
data_table = pd.concat([data_table,age], axis=1)
data_table['cat_age'] = pd.cut(data_table.age, bins=[0, 25, 30, 60, float('Inf')], labels=['teen', 'young','adult', 'old'])

table = data_table.pivot_table('purchased', 'cat_age')
table['% purchased'] = (table['purchased']/table['purchased'].sum())*100
table

,purchased,% purchased
cat_age,,
teen,0.000000,0.000000
young,0.061947,4.252498
adult,0.461438,31.676540
old,0.933333,64.070961


In [416]:
table = data_table.pivot_table('purchased', 'cat_wage')
table['% purchased'] = (table['purchased']/table['purchased'].sum())*100
table

,purchased,% purchased
cat_wage,,
medium,0.443946,30.835831
medium-high,0.256228,17.797194
high,0.739535,51.366974


In [417]:
table = data_table.pivot_table('purchased', 'gender_Male')
table['% purchased'] = (table['purchased']/table['purchased'].sum())*100
table

,purchased,% purchased
gender_Male,,
0.0,0.423372,52.787327
1.0,0.378661,47.212677


In [418]:
table = data_table.pivot_table('purchased', ['gender_Male','cat_wage','cat_age'])
table['% purchased'] = (table['purchased']/table['purchased'].sum())*100
table

purchased  % purchased
gender_Male cat_wage    cat_age                        
0.0         medium      teen      0.000000     0.000000
                        young     0.000000     0.000000
                        adult     0.586207     6.182773
                        old       0.800000     8.437666
            medium-high teen      0.000000     0.000000
                        young     0.090909     0.958826
                        adult     0.295000     3.111389
                        old       0.888889     9.375185
            high        teen      0.000000     0.000000
                        young     0.111111     1.171898
                        adult     0.756522     7.979098
                        old       1.000000    10.547083
1.0         medium      teen      0.000000     0.000000
                        young     0.000000     0.000000
                        adult     0.492754     5.197114
                        old       1.000000    10.547083
            medium-high teen      0.000000     0.000000
                        young     0.000000     0.000000
                        adult     0.290749     3.066553
                        old       1.000000    10.547083
            high        teen      0.000000     0.000000
                        young     0.333333     3.515694
                        adult     0.835821     8.815473
                        old       1.000000    10.547083

In [419]:
data_table.pivot_table('wage','cat_wage',aggfunc=['mean','median'])

,mean,median
,wage,wage
cat_wage,,
medium,33663.117188,33500.0
medium-high,75678.828125,76500.0
high,121865.695312,117500.0


In [420]:
data_table.pivot_table('wage','gender_Male',aggfunc=['mean','median'])

,mean,median
,wage,wage
gender_Male,,
0.0,77286.640625,77500.0
1.0,75095.976562,75500.0
